# Deep Learning for Media final Project

### Weather Condition Classification Using Image Recognition

---
Weather Condition Classification Using Image Recognition

Part 1: Organizing our Data, Rita

Part 2: Building, training, and evaluating baseline,Rita

Part 3: Regularize the model,

Part 4: Building and using a pertained model,

Part 5: Data Analysis,

Part 6: Confusion Matrix



### Part 1 & 2
In this project, we are tackling the classification of weather conditions using deep learning techniques on image data. Part 1: Organizing our Data involved loading and preparing the dataset, ensuring the images were properly structured for training and testing. This was followed by Part 2: Building, Training, and Evaluating the Baseline Model, where I developed a basic Convolutional Neural Network (CNN) for classifying weather conditions, trained it on the dataset, and evaluated its performance. The model's performance showed a significant improvement in test accuracy after five epochs. Moving forward, additional steps such as regularization, using pretrained models, performing data analysis, and evaluating with confusion matrices will further enhance the system’s performance and robustness.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Change the path to point to the folder in your Google Drive
data_dir = '/content/drive/MyDrive/dl4m/dl4m-group7-main/data'


In [3]:
# main.py or a Colab cell

import torch
from project import load_data  # Import your data loading function
from utils import BaselineCNN, train_model, evaluate_model  # Import model and helper functions

# Mount Google Drive to access data (only needed in Colab)
from google.colab import drive
drive.mount('/content/drive')

# Set device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load data (make sure to update the data path accordingly to your Google Drive structure)
data_dir = '/content/drive/MyDrive/dl4m/dl4m-group7-main/data'  # Update this with your folder path in Google Drive
train_loader, test_loader, classes = load_data(data_dir=data_dir)
num_classes = len(classes)

# Initialize model, loss function, and optimizer
model = BaselineCNN(num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate the model
num_epochs = 5  # You can adjust the number of epochs here
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Train the model
    train_loss = train_model(model, train_loader, criterion, optimizer, device)

    # Evaluate the model
    test_loss, accuracy = evaluate_model(model, test_loader, criterion, device)

    print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | Accuracy: {accuracy:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/5


Evaluating: 100%|██████████| 8/8 [00:36<00:00,  4.59s/it]


Train Loss: 0.8289 | Test Loss: 0.6303 | Accuracy: 81.89%
Epoch 2/5


Evaluating: 100%|██████████| 8/8 [00:02<00:00,  3.57it/s]


Train Loss: 0.2734 | Test Loss: 0.3187 | Accuracy: 90.12%
Epoch 3/5


...

Train Loss: 0.1857 | Test Loss: 0.2856 | Accuracy: 91.77%
Epoch 4/5


Evaluating: 100%|██████████| 8/8 [00:02<00:00,  3.61it/s]


Train Loss: 0.1346 | Test Loss: 0.2173 | Accuracy: 93.42%
Epoch 5/5


Evaluating: 100%|██████████| 8/8 [00:02<00:00,  3.02it/s]

Train Loss: 0.0926 | Test Loss: 0.2825 | Accuracy: 91.36%
